# Residual Bloklar: Wide, Pre-Act, Post-Act, Bottleneck(Expansion), ResNeXt Grouped

Bu notebook; aşağıdaki residual blok varyantlarını **kısa ama net** şekilde açıklar ve her biri için **PyTorch örnek implementasyon iskeleti** verir:
- **Wide** (Wide ResNet yaklaşımı)
- **Post-Activation** (orijinal ResNet bloğu)
- **Pre-Activation** (ResNet v2)
- **Bottleneck + Expansion** (ResNet-50/101 tarzı)
- **ResNeXt Grouped Bottleneck** (cardinality / grouped conv)

> Not: Kodlar “notluk” yazıldı: okunabilirlik ve kavramı oturtma amaçlıdır.

## 1) Kısa Kavramlar
- **Residual/skip connection:** `y = F(x) + x` (boyutlar uyuyorsa).
- **Projection shortcut:** Boyut/kanal değişiyorsa `1x1 conv` ile `x` dönüştürülür.
- **Activation yeri:** Pre-Act vs Post-Act farkının özü burasıdır.
- **Expansion:** Bottleneck blokta çıkış kanalını büyütme oranı (genelde `4`).
- **Groups:** Grouped convolution ile kanal gruplarını ayrı ayrı işler (ResNeXt).

## 2) Wide Residual Block (Wide ResNet yaklaşımı)
**Temel fikir:** Derinliği artırmak yerine **kanal genişliğini (width)** artır.

**Ne değişir?**
- Aynı blok yapısı korunabilir (basic residual), ama `out_channels` daha büyük seçilir.
- Çoğu pratikte: daha az derinlik + daha geniş katmanlar → daha stabil eğitim.

**Artı/Eksi:**
- ✅ Daha stabil/kolay optimize
- ❌ Parametre ve hesap maliyeti artar

## 3) Post-Activation Residual Block (ResNet v1)
**Akış (tipik):**
1) `Conv → BN → ReLU → Conv → BN`
2) `+ shortcut`
3) `ReLU` (toplamadan sonra)

**Öz:** Aktivasyon **toplamadan sonra** gelir. Orijinal ResNet tasarımı bu şekildedir.

## 4) Pre-Activation Residual Block (ResNet v2)
**Akış (tipik):**
1) `BN → ReLU → Conv → BN → ReLU → Conv`
2) `+ shortcut`

**Öz:** BN+ReLU **konvolüsyondan önce** gelir; toplama sonrası ayrıca ReLU şart değildir.

**Neden önemli?**
- Skip path üzerinden gradient daha “temiz” akar.
- Çok derin ağlarda optimizasyonu kolaylaştırır.

## 5) Bottleneck (with Expansion)
**Amaç:** Hesap maliyetini düşürürken temsil gücünü korumak.

**3 katmanlı yapı:**
- `1x1` **reduce** (kanalı düşür)
- `3x3` **process**
- `1x1` **expand** (kanalı büyüt)

**Expansion:** Çıkış kanalını `bottleneck_ch * expansion` yapar. ResNet-50/101’de çoğunlukla `expansion=4`.

**Örnek:**
- bottleneck_ch=64, expansion=4 → block çıkışı 256 kanal.

## 6) ResNeXt Grouped Residual Conv (Grouped Bottleneck)
**ResNeXt fikri:** Bottleneck’in ortasındaki `3x3` katmanı **grouped convolution** yap.

**Cardinality (groups):**
- Derinlik/width yerine “grup sayısını” artırarak kapasiteyi büyütür.

**Öz:**
- `1x1 reduce` → `3x3 grouped conv (groups=C)` → `1x1 expand`

**Artı/Eksi:**
- ✅ Benzer FLOPs ile daha güçlü temsil (pratikte iyi)
- ❌ Grouped conv donanım/implementasyona göre değişken hız

## 7) Tek Bakışta Özet Tablo

| Blok | Aktivasyon Konumu | İç Yapı | Boyut Eşleme | Tipik Kullanım | Ana Fikir |
|---|---|---|---|---|---|
| **Wide** | (bloka bağlı) | Genelde Basic/PreAct Basic | Gerekirse 1x1 projection | WideResNet varyantları | Derinlik yerine **kanal genişliği** |
| **Post-Act (v1)** | Toplamadan **sonra** ReLU | Conv-BN-ReLU-Conv-BN + Add + ReLU | 1x1 projection opsiyonel | ResNet-18/34/50 (orijinal) | Klasik residual akış |
| **Pre-Act (v2)** | Conv’lardan **önce** BN+ReLU | BN-ReLU-Conv-BN-ReLU-Conv + Add | 1x1 projection opsiyonel (çoğu projeksiyon da preact’e uyar) | ResNet v2, çok derin ağlar | Daha temiz gradient akışı |
| **Bottleneck + Expansion** | v1 veya v2 tasarımla | 1x1 reduce → 3x3 → 1x1 expand | Sık kullanılır (stride/kanal değişimi) | ResNet-50/101/152 | Ucuz hesapla güçlü temsil |
| **ResNeXt Grouped** | v1 veya v2 tasarımla | 1x1 → 3x3 **groups** → 1x1 | Aynı | ResNeXt-50/101 | **Cardinality** ile kapasite |


## 8) PyTorch: Ortak Yardımcılar
Aşağıda blokları yazarken tekrar etmemek için küçük yardımcı fonksiyonlar var.

In [1]:
import torch
import torch.nn as nn
from typing import Optional, Callable

def conv3x3(in_ch: int, out_ch: int, stride: int = 1, groups: int = 1) -> nn.Conv2d:
    # 3x3 conv: padding=1 ile uzamsal boyut korunur (stride=1 ise)
    return nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=stride, padding=1, bias=False, groups=groups)

def conv1x1(in_ch: int, out_ch: int, stride: int = 1) -> nn.Conv2d:
    # 1x1 conv: kanal eşleme / projeksiyon için
    return nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=stride, padding=0, bias=False)


## 9) Post-Activation Basic Residual Block (ResNet v1)
Bu, ResNet-18/34 tarzı **basic** bloktur.
- Aktivasyon: **Add’den sonra** ReLU
- Boyut değişirse: shortcut için `1x1 conv` projeksiyon.

In [2]:
class BasicBlockPostAct(nn.Module):
    expansion = 1  # basic block çıkış kanalı = out_ch * expansion

    def __init__(self, in_ch: int, out_ch: int, stride: int = 1):
        super().__init__()

        self.conv1 = conv3x3(in_ch, out_ch, stride=stride)
        self.bn1   = nn.BatchNorm2d(out_ch)
        self.relu  = nn.ReLU(inplace=True)

        self.conv2 = conv3x3(out_ch, out_ch, stride=1)
        self.bn2   = nn.BatchNorm2d(out_ch)

        # shortcut: boyut/kanal değişiyorsa 1x1 projection gerekir
        self.downsample = None
        if stride != 1 or in_ch != out_ch:
            self.downsample = nn.Sequential(
                conv1x1(in_ch, out_ch, stride=stride),
                nn.BatchNorm2d(out_ch),
            )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out = out + identity
        out = self.relu(out)  # <-- Post-Act farkı: ReLU burada
        return out


## 10) Pre-Activation Basic Residual Block (ResNet v2)
Bu da basic block ama **BN+ReLU conv'dan önce**.
- Çoğu tasarımda add sonrası ekstra ReLU gerekmez.
- Downsample (projection) yapılacaksa pre-activation çıkışından beslemek yaygındır.

In [3]:
class BasicBlockPreAct(nn.Module):
    expansion = 1

    def __init__(self, in_ch: int, out_ch: int, stride: int = 1):
        super().__init__()

        self.bn1   = nn.BatchNorm2d(in_ch)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = conv3x3(in_ch, out_ch, stride=stride)

        self.bn2   = nn.BatchNorm2d(out_ch)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_ch, out_ch, stride=1)

        self.downsample = None
        if stride != 1 or in_ch != out_ch:
            self.downsample = conv1x1(in_ch, out_ch, stride=stride)  # BN opsiyonel

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x

        out = self.bn1(x)
        out = self.relu1(out)

        # projection shortcut genelde burada preact'ten beslenir
        if self.downsample is not None:
            identity = self.downsample(out)

        out = self.conv1(out)

        out = self.bn2(out)
        out = self.relu2(out)
        out = self.conv2(out)

        out = out + identity
        return out


## 11) Bottleneck (with Expansion) - Post-Act örneği
ResNet-50/101 tarzı.
- `1x1 reduce` → `3x3` → `1x1 expand`
- `expansion=4` tipiktir.
- Aktivasyon v1 tarzı: add sonrası ReLU.

In [4]:
class BottleneckPostAct(nn.Module):
    expansion = 4

    def __init__(self, in_ch: int, bottleneck_ch: int, stride: int = 1):
        super().__init__()
        out_ch = bottleneck_ch * self.expansion

        self.conv1 = conv1x1(in_ch, bottleneck_ch, stride=1)
        self.bn1   = nn.BatchNorm2d(bottleneck_ch)

        self.conv2 = conv3x3(bottleneck_ch, bottleneck_ch, stride=stride)
        self.bn2   = nn.BatchNorm2d(bottleneck_ch)

        self.conv3 = conv1x1(bottleneck_ch, out_ch, stride=1)
        self.bn3   = nn.BatchNorm2d(out_ch)

        self.relu  = nn.ReLU(inplace=True)

        self.downsample = None
        if stride != 1 or in_ch != out_ch:
            self.downsample = nn.Sequential(
                conv1x1(in_ch, out_ch, stride=stride),
                nn.BatchNorm2d(out_ch),
            )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x

        out = self.conv1(x); out = self.bn1(out); out = self.relu(out)
        out = self.conv2(out); out = self.bn2(out); out = self.relu(out)
        out = self.conv3(out); out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out = out + identity
        out = self.relu(out)
        return out


## 12) ResNeXt Grouped Bottleneck - Post-Act örneği
Bottleneck ile aynı mantık, fark: ortadaki 3x3 conv **groups** ile çalışır.

ResNeXt’te genelde şu parametrizasyon görülür:
- `groups` = cardinality
- `width_per_group` ile effective bottleneck width ayarlanır

Aşağıdaki implementasyon basit tutuldu: `groups` direkt veriliyor.

In [5]:
class ResNeXtBottleneckPostAct(nn.Module):
    expansion = 4

    def __init__(self, in_ch: int, bottleneck_ch: int, stride: int = 1, groups: int = 32):
        super().__init__()
        out_ch = bottleneck_ch * self.expansion

        # 1x1 reduce
        self.conv1 = conv1x1(in_ch, bottleneck_ch, stride=1)
        self.bn1   = nn.BatchNorm2d(bottleneck_ch)

        # 3x3 grouped conv
        self.conv2 = conv3x3(bottleneck_ch, bottleneck_ch, stride=stride, groups=groups)
        self.bn2   = nn.BatchNorm2d(bottleneck_ch)

        # 1x1 expand
        self.conv3 = conv1x1(bottleneck_ch, out_ch, stride=1)
        self.bn3   = nn.BatchNorm2d(out_ch)

        self.relu  = nn.ReLU(inplace=True)

        self.downsample = None
        if stride != 1 or in_ch != out_ch:
            self.downsample = nn.Sequential(
                conv1x1(in_ch, out_ch, stride=stride),
                nn.BatchNorm2d(out_ch),
            )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x

        out = self.conv1(x); out = self.bn1(out); out = self.relu(out)
        out = self.conv2(out); out = self.bn2(out); out = self.relu(out)
        out = self.conv3(out); out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out = out + identity
        out = self.relu(out)
        return out


## 13) Wide yaklaşımını koda bağlamak
**Wide** ayrı bir blok değildir; genelde blokların `out_ch` değerini büyütmektir.

Aşağıdaki örnek: BasicBlockPostAct ile normalde `out_ch=64` kullanacaksan, Wide için `k * 64` (ör. `k=2,4,8`) kullanırsın.

In [6]:
# Örnek: Wide factor ile kanal büyütme
wide_factor = 4
block = BasicBlockPostAct(in_ch=64, out_ch=64 * wide_factor, stride=1)

x = torch.randn(2, 64, 56, 56)
y = block(x)
y.shape


torch.Size([2, 256, 56, 56])

## 14) Hızlı sanity-check
Aşağıdaki hücre tüm blokların çalıştığını hızlıca kontrol eder.

In [7]:
x = torch.randn(2, 64, 56, 56)

m1 = BasicBlockPostAct(64, 64, stride=1)
m2 = BasicBlockPreAct(64, 64, stride=1)
m3 = BottleneckPostAct(64, bottleneck_ch=64, stride=1)  # çıkış 256 kanal
m4 = ResNeXtBottleneckPostAct(64, bottleneck_ch=64, stride=1, groups=32)  # çıkış 256 kanal

y1 = m1(x)
y2 = m2(x)
y3 = m3(x)
y4 = m4(x)

(y1.shape, y2.shape, y3.shape, y4.shape)

(torch.Size([2, 64, 56, 56]),
 torch.Size([2, 64, 56, 56]),
 torch.Size([2, 256, 56, 56]),
 torch.Size([2, 256, 56, 56]))